In [1]:
import pymongo
import pandas as pd
from pandas import Series,DataFrame
import matplotlib as plt
import matplotlib.dates as mdate
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures
fx = futures.fx
ore62 = futures.ore62
ore_market = futures.ore_market

#加载数据
fx = DataFrame(list(fx.find()))
ore62 = DataFrame(list(ore62.find()))
ore_market = DataFrame(list(ore_market.find()))

trade=pd.merge(ore_market,ore62, on=['date'],how='left')
trade=trade.dropna()
del trade['_id_x']
del trade['_id_y']
trade=pd.merge(trade,fx, on=['date'],how='left')
del trade['_id']
trade=trade.set_index('date')
trade.sort_index(inplace=True)
trade
trade['折算盘面']=trade.apply(lambda x: (x['62%ore']*x['CNY']*1.17+35),axis=1)
trade['升贴水']=trade.apply(lambda x: (x['ore_main']-(x['62%ore']*x['CNY']*1.17+35))/x['ore_main']*100,axis=1)
trade
trade.to_csv(r"c:\ore.csv",mode='a',encoding='ANSI',header=True)
# trade.plot.line(x='date',y='净持仓价值',figsize=(20,16),title=members)

KeyError: 'date'